In [134]:
import syft as sy
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from syft.core.adp.entity import Entity
import torch

In [8]:
ds_domain = sy.login(
    email = "ttomek.koziak@gmail.com",
    password = "1234567890",
    port = 8081
)

Connecting to None... done! 	 Logging into tomek... done!


In [9]:
ds_domain.privacy_budget

10.0

In [10]:
ds_domain.datasets

In [163]:
# Load data
"""Load unsw dataset"""
unsw = pd.read_csv("UNSW_NB15_training-set.csv")
y = unsw.iloc[:,-1:]
#one-hot-encode parameters
proto = pd.get_dummies(unsw['proto'])
state = pd.get_dummies(unsw['state'])
service = pd.get_dummies(unsw['service'])
service = service.iloc[:,1:]
raw_data = unsw.values

#remove encoded parameters and add one hot
for x in ['proto', 'state', 'service','label','attack_cat']:
    unsw = unsw.drop(x,axis = 1)
    
for x in [proto, state, service]:
    unsw = unsw.join(x)
    
X = unsw.iloc[:,:-2]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [164]:
X = X.join(y)
X = X.head(100)

In [165]:
X

,id,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,...,dns,ftp,ftp-data,http,irc,pop3,radius,smtp,snmp,label
0,1,0.000011,2,0,496,0,90909.090200,254,0,1.803636e+08,...,0,0,0,0,0,0,0,0,0,0
1,2,0.000008,2,0,1762,0,125000.000300,254,0,8.810000e+08,...,0,0,0,0,0,0,0,0,0,0
2,3,0.000005,2,0,1068,0,200000.005100,254,0,8.544000e+08,...,0,0,0,0,0,0,0,0,0,0
3,4,0.000006,2,0,900,0,166666.660800,254,0,6.000000e+08,...,0,0,0,0,0,0,0,0,0,0
4,5,0.000010,2,0,2126,0,100000.002500,254,0,8.504000e+08,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,16.044233,12,0,540,0,0.685605,254,0,2.468177e+02,...,0,0,0,0,0,0,0,0,0,0
96,97,0.664512,10,6,534,268,22.572957,254,252,5.790716e+03,...,0,0,0,0,0,0,0,0,0,0
97,98,0.860208,10,6,534,268,17.437643,254,252,4.473337e+03,...,0,0,0,0,0,0,0,0,0,0
98,99,15.102544,10,0,450,0,0.595926,254,0,2.145334e+02,...,0,0,0,0,0,0,0,0,0,0


In [166]:
# Convert a Pandas Dataframe to NumPy array
print(f'raw_data is of type: {type(X)}')
raw_data = X.values
print(f'raw_data is of type: {type(raw_data)}')

raw_data is of type: <class 'pandas.core.frame.DataFrame'>
raw_data is of type: <class 'numpy.ndarray'>


In [167]:
print(f'random_data is of dtype: {raw_data.dtype}')
raw_data_32 = raw_data.astype(np.int32)
print(f'random_data is now of type: {raw_data_32.dtype}')

random_data is of dtype: float64
random_data is now of type: int32


In [168]:
# select all of Country 0's data
packet0_data = raw_data[0, :]

In [169]:
np.min(raw_data)

0.0

In [170]:
np.max(raw_data)

4280658180.0

In [171]:
packet0_data

array([1.00000000e+00, 1.10000000e-05, 2.00000000e+00, 0.00000000e+00,
       4.96000000e+02, 0.00000000e+00, 9.09090902e+04, 2.54000000e+02,
       0.00000000e+00, 1.80363632e+08, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.10000000e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.48000000e+02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.00000000e+00, 2.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 2.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.00000000e+00, 2.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [172]:
# Specify it to be a np.int32 dtype
packet0_data = packet0_data.astype(np.int32)

In [173]:
packet0_data

array([        1,         0,         2,         0,       496,         0,
           90909,       254,         0, 180363632,         0,         0,
               0,         0,         0,         0,         0,         0,
               0,         0,         0,         0,         0,         0,
             248,         0,         0,         0,         2,         2,
               1,         1,         1,         2,         0,         0,
               0,         1,         2,         0,         0,         0,
               0,         0,         0,         0,         0,         0,
               0,         0,         0,         0,         0,         0,
               0,         0,         0,         0,         0,         0,
               0,         0,         0,         0,         0,         0,
               0,         0,         0,         0,         0,         0,
               0,         0,         0,         0,         0,         0,
               0,         0,         0,         0, 

In [174]:
packet0 = Entity(name="packet 0")

# Create a Syft tensor
data = sy.Tensor(packet0_data)

# Add DP metadata
data = data.private(min_val=0, max_val=4280658180, entities=packet0)

In [175]:
data

Tensor(child=SingleEntityPhiTensor(entity=packet 0, child=[        1         0         2         0       496         0     90909
       254         0 180363632         0         0         0         0
         0         0         0         0         0         0         0
         0         0         0       248         0         0         0
         2         2         1         1         1         2         0
         0         0         1         2         0         0         0
         0         0         0         0         0         0         0
         0         0         0         0         0         0         0
         0         0         0         0         0         0         0
         0         0         0         0         0         0         0
         0         0         0         0         0         0         0
         0         0         0         0         0         0         0
         0         0         0         0         0         0         0
         0         

In [176]:
dataset = dict()
entities = []
for i in range(raw_data.shape[0]):
    packet_name = f"Packet {i}"
    
    # Create a new Entity correspoinding to the country and add it to the list
    new_entity = Entity(name=packet_name)
    entities.append(new_entity)
    
    # Add it to the Dataset Dictionary
    dataset[packet_name] = sy.Tensor(raw_data[i, :].astype(np.int32)).private(min_val=0, max_val=4280658180, entities=new_entity)

In [177]:
dataset

{'Packet 0': Tensor(child=SingleEntityPhiTensor(entity=Packet 0, child=[        1         0         2         0       496         0     90909
        254         0 180363632         0         0         0         0
          0         0         0         0         0         0         0
          0         0         0       248         0         0         0
          2         2         1         1         1         2         0
          0         0         1         2         0         0         0
          0         0         0         0         0         0         0
          0         0         0         0         0         0         0
          0         0         0         0         0         0         0
          0         0         0         0         0         0         0
          0         0         0         0         0         0         0
          0         0         0         0         0         0         0
          0         0         0         0         0         0     

In [178]:
# Login to Domain Node
domain_node = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into tomek... done!


In [179]:
test = dict()
test[0] = test_data

In [180]:
# Upload the dataset!
domain_node.load_dataset(
    assets=dataset, 
    name="Packets", 
    description="Iot packets, last element label", 
    metadata="iot"
)

Loading dataset... uploading... SUCCESS!                                                                                                                                      

Run <your client variable>.datasets to see your new dataset loaded into your machine!


In [182]:
domain_node.datasets

Idx,Name,Description,Assets,Id
[0],test,test,"[""0""] -> Tensor",d36bdb58-6cb3-4697-9f4f-0e1d84342a5a
[1],Packets,"Iot packets, last element label","[""Packet 0""] -> Tensor[""Packet 1""] -> Tensor[""Packet 2""] -> Tensor...",c55a8adb-804d-48d9-a9a1-77904a2dfc65
[2],Packets,"Iot packets, last element label","[""Packet 0""] -> Tensor[""Packet 1""] -> Tensor[""Packet 2""] -> Tensor...",b9c96199-87a1-4adf-9bbd-391a491a1559


In [183]:
domain_node.datasets[2]

Dataset: Packets
Description: Iot packets, last element label


 assets = my_dataset.assets 

to view receive a dictionary you can parse through using Python
(as opposed to blowing up your notebook with a massive printed table).



Asset Key,Type,Shape
"[""Packet 0""]",Tensor,"(189,)"
"[""Packet 1""]",Tensor,"(189,)"
"[""Packet 2""]",Tensor,"(189,)"
"[""Packet 3""]",Tensor,"(189,)"
"[""Packet 4""]",Tensor,"(189,)"
"[""Packet 5""]",Tensor,"(189,)"
"[""Packet 6""]",Tensor,"(189,)"
"[""Packet 7""]",Tensor,"(189,)"
"[""Packet 8""]",Tensor,"(189,)"
"[""Packet 9""]",Tensor,"(189,)"
